# Testing things in the notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import sys
%matplotlib inline
sys.version

Import the necessary Libraries 

In [ ]:
from pyworkout.parsers import tcxtools

How bad does one of the TCX files look? The data cleaning will be done by the framework, but look how bad it is in raw form

In [ ]:
workout_data = tcxtools.TCXPandas('raw_data/activity_248017020.tcx') # Create the Class Object
workout_data_dd = workout_data.parse()

## Explore and visualize some of the data

Free form here: play with pandas a bit to see what is possible.  Note that you can actually instantiate pandas graphing using matplotlib here

In [ ]:
workout_data_dd.plot.scatter('cadence','power')

## Playing with Bokeh

In [ ]:
import bokeh

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()

p = figure(title="Exploring power vs distance", x_axis_label='x', y_axis_label='y')
p.line(workout_data_dd.distance, workout_data_dd.power, legend="Power (Watts)", line_width=2)

# show the results
show(p)

In [ ]:
output_notebook()
p = figure(title="Exploring power vs cadence", x_axis_label='x', y_axis_label='y')
p.scatter(workout_data_dd.power, workout_data_dd.cadence)

# show the results
show(p)

## Scikit-learn: some quick fun testing

Import the basic model necessities

In [ ]:
from sklearn import model_selection
from sklearn import linear_model

Parse the dataset, then add in model_selection's train/test split
before training the model, drop the following features: time, power, latitude, longitude

We are trying to calculate the power (target) based on the other features

In [ ]:
dataset = workout_data.parse()
# Reduce the dataset
red_dataset = dataset.drop(['time','power','latitude','longitude'], axis=1)
X_train, X_test, y_train, y_test = # TODO

In [ ]:
regression_model = linear_model.LinearRegression()
# TODO: Train your model
mse = np.mean((regression_model.predict(X_test) - y_test) ** 2)
score = regression_model.score(X_test, y_test)
print(regression_model.coef_, mse, score, regression_model)

In [ ]:
pred_values = regression_model.predict(dataset.drop(['time','power','latitude','longitude'], axis=1))
plt.plot(dataset.distance[5000:6000], dataset.power[5000:6000], c='red')
plt.plot(dataset.distance[5000:6000], pred_values[5000:6000])
plt.show()

In [ ]:
red_dataset.columns

# Scaling out via Dask

One of the neat things of dask is when you need general analysis in concurrent flows to be accomplished
What is the code doing below?

In [ ]:
import dask.dataframe as dd
from distributed import client
from dask import delayed
tcx_list = glob.glob("raw_data/*.tcx")
list_of_delayed = []
for files in tcx_list:
    list_of_delayed.append(delayed(tcxtools.TCXPandas(files).parse()))

In [ ]:
list_of_delayed

In [ ]:
total = dd.from_delayed(list_of_delayed)

## Run the same analysis of ML as before, but now with this dask dataframe delayed